<a href="https://colab.research.google.com/github/hitallocavas/cdc-results/blob/main/tg-cdc-results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
# Importação dos Dados

import json

def import_file(filename):
  with open(filename, 'r') as f:
    return json.load(f)

In [20]:
# Converter em DataFrame

import pandas as pd

def convert_to_df(data):
  return pd.json_normalize(data)

In [21]:
def normalize_mongodb(df):
  df['createdAt'] = pd.to_datetime(df['createdAt.$date'], format='mixed', utc=True)
  df['updatedAt'] = pd.to_datetime(df['updatedAt.$date'], format='mixed', utc=True)
  df['amount'] = df['amount.$numberDecimal'];
  df.drop(columns=['amount.$numberDecimal'], inplace=True)

In [22]:
import numpy as np

def compare_dataframe_fields(df1, df2, key='transactionId', fields=[], float_tolerance=0.01):
    df = df1.merge(df2, on=key, suffixes=('_pg', '_mongo'))

    df['amount_pg'] = df['amount_pg'].astype(float)
    df['amount_mongo'] = df['amount_mongo'].astype(float)

    comparisons = {}
    for field in fields:
        col_pg = f'{field}_pg'
        col_mongo = f'{field}_mongo'

        if np.issubdtype(df[col_pg].dtype, np.floating):
            comparisons[field] = np.isclose(df[col_pg], df[col_mongo], atol=float_tolerance)
        else:
            comparisons[field] = df[col_pg] == df[col_mongo]

        df[f'{field}_match'] = comparisons[field]

    df['record_match'] = df[[f'{field}_match' for field in fields]].all(axis=1)

    return df

In [23]:
def processar_cenario(nome_arquivo_mongo, nome_arquivo_postgres, descricao_filtro='Cenário 1 - Req'):
    # Importa e prepara os dataframes
    df_mongo = convert_to_df(import_file(nome_arquivo_mongo))
    normalize_mongodb(df_mongo)
    df_postgres = convert_to_df(import_file(nome_arquivo_postgres))

    # Filtra pelas descrições desejadas
    df_mongo = df_mongo[df_mongo['description'].str.contains(descricao_filtro, case=False, na=False)]
    df_postgres = df_postgres[df_postgres['description'].str.contains(descricao_filtro, case=False, na=False)]

    # Compara os campos especificados
    comparado = compare_dataframe_fields(
        df_postgres,
        df_mongo,
        fields=['type', 'clientId', 'amount', 'description']
    )

    # Calcula igualdade de campos
    total = len(comparado)
    campos_match = {}
    for campo in ['clientId', 'type', 'amount', 'description']:
        iguais = comparado[f'{campo}_match'].sum()
        campos_match[campo] = {
            'iguais': int(iguais),
            'total': int(total),
            'percentual': float((iguais / total) * 100)
        }

    # Calcula latência de criação
    comparado['createdAt_pg'] = pd.to_datetime(comparado['createdAt_pg'], format='mixed', utc=True).dt.tz_convert(None)
    comparado['createdAt_mongo'] = pd.to_datetime(comparado['createdAt_mongo'], format='mixed', utc=True).dt.tz_convert(None)
    comparado['latency_creation'] = (comparado['createdAt_mongo'] - comparado['createdAt_pg']).dt.total_seconds() * 1000

    # Calcula latência de atualização
    comparado['updatedAt_pg'] = pd.to_datetime(comparado['updatedAt_pg'], format='mixed', utc=True).dt.tz_convert(None)
    comparado['updatedAt_mongo'] = pd.to_datetime(comparado['updatedAt_mongo'], format='mixed', utc=True).dt.tz_convert(None)
    comparado['latency_update'] = (comparado['updatedAt_mongo'] - comparado['updatedAt_pg']).dt.total_seconds() * 1000

    # Métricas de consistência
    total_registros = len(comparado)
    total_consistentes = comparado['record_match'].sum()
    total_inconsistentes = total_registros - total_consistentes
    taxa_consistencia = (total_consistentes / total_registros) * 100

    # Resumo das latências
    def calcular_resumo_latencia(serie):
        return {
            'Média (ms)': serie.mean(),
            'Mínimo (ms)': serie.min(),
            'Máximo (ms)': serie.max(),
            'p90 (ms)': serie.quantile(0.90),
            'p95 (ms)': serie.quantile(0.95)
        }

    resumo_create = calcular_resumo_latencia(comparado['latency_creation'])
    resumo_update = calcular_resumo_latencia(comparado['latency_update'])

    # Retorna tudo em um dicionário
    return {
        'comparado': comparado,
        'igualdade_campos': campos_match,
        'total_registros': int(total_registros),
        'total_consistentes': int(total_consistentes),
        'total_inconsistentes': int(total_inconsistentes),
        'taxa_consistencia': float(taxa_consistencia),
        'latencia_create': resumo_create,
        'latencia_update': resumo_update,
        'inconsistentes': comparado[~comparado['record_match']]
    }


In [24]:
def print_resultados(resultados):
    print("==== RESULTADOS DA COMPARAÇÃO ====\n")

    print(f"Total de registros comparados: {resultados['total_registros']}")
    print(f"Registros consistentes: {resultados['total_consistentes']}")
    print(f"Registros inconsistentes: {resultados['total_inconsistentes']}")
    print(f"Taxa de consistência: {resultados['taxa_consistencia']:.2f}%\n")

    print("==== IGUALDADE DOS CAMPOS ====")
    for campo, dados in resultados['igualdade_campos'].items():
        print(f"{campo}: {dados['iguais']} iguais de {dados['total']} "
              f"({dados['percentual']:.2f}%)")
    print()

    print("==== LATÊNCIA DE CRIAÇÃO ====")
    for chave, valor in resultados['latencia_create'].items():
        print(f"{chave}: {valor:.2f} ms")
    print()

    print("==== LATÊNCIA DE ATUALIZAÇÃO ====")
    for chave, valor in resultados['latencia_update'].items():
        print(f"{chave}: {valor:.2f} ms")
    print()

    if not resultados['inconsistentes'].empty:
        print("==== EXEMPLO DE REGISTROS INCONSISTENTES ====")
        display(resultados['inconsistentes'].head())
    else:
        print("Todos os registros estão consistentes.\n")

In [28]:
resultados_cenario1 = processar_cenario('mongodb.json', 'postgres.json')
print_resultados(resultados_cenario1)

==== RESULTADOS DA COMPARAÇÃO ====

Total de registros comparados: 68
Registros consistentes: 68
Registros inconsistentes: 0
Taxa de consistência: 100.00%

==== IGUALDADE DOS CAMPOS ====
clientId: 68 iguais de 68 (100.00%)
type: 68 iguais de 68 (100.00%)
amount: 68 iguais de 68 (100.00%)
description: 68 iguais de 68 (100.00%)

==== LATÊNCIA DE CRIAÇÃO ====
Média (ms): 479.97 ms
Mínimo (ms): 91.13 ms
Máximo (ms): 924.12 ms
p90 (ms): 657.65 ms
p95 (ms): 711.79 ms

==== LATÊNCIA DE ATUALIZAÇÃO ====
Média (ms): 579.89 ms
Mínimo (ms): 209.78 ms
Máximo (ms): 919.88 ms
p90 (ms): 840.38 ms
p95 (ms): 856.26 ms

Todos os registros estão consistentes.



In [26]:
resultados_cenario2 = processar_cenario('mongodb.json', 'postgres.json', "Cenário 2 - Req")
print_resultados(resultados_cenario2)

==== RESULTADOS DA COMPARAÇÃO ====

Total de registros comparados: 290
Registros consistentes: 290
Registros inconsistentes: 0
Taxa de consistência: 100.00%

==== IGUALDADE DOS CAMPOS ====
clientId: 290 iguais de 290 (100.00%)
type: 290 iguais de 290 (100.00%)
amount: 290 iguais de 290 (100.00%)
description: 290 iguais de 290 (100.00%)

==== LATÊNCIA DE CRIAÇÃO ====
Média (ms): 542.25 ms
Mínimo (ms): 184.92 ms
Máximo (ms): 967.31 ms
p90 (ms): 787.02 ms
p95 (ms): 864.16 ms

==== LATÊNCIA DE ATUALIZAÇÃO ====
Média (ms): 546.10 ms
Mínimo (ms): 88.39 ms
Máximo (ms): 965.98 ms
p90 (ms): 793.28 ms
p95 (ms): 846.56 ms

Todos os registros estão consistentes.



In [27]:
resultados_cenario3 = processar_cenario('mongodb.json', 'postgres.json', "Cenário 3 - Req")
print_resultados(resultados_cenario3)

==== RESULTADOS DA COMPARAÇÃO ====

Total de registros comparados: 273
Registros consistentes: 273
Registros inconsistentes: 0
Taxa de consistência: 100.00%

==== IGUALDADE DOS CAMPOS ====
clientId: 273 iguais de 273 (100.00%)
type: 273 iguais de 273 (100.00%)
amount: 273 iguais de 273 (100.00%)
description: 273 iguais de 273 (100.00%)

==== LATÊNCIA DE CRIAÇÃO ====
Média (ms): 486.79 ms
Mínimo (ms): 35.72 ms
Máximo (ms): 941.69 ms
p90 (ms): 744.14 ms
p95 (ms): 835.27 ms

==== LATÊNCIA DE ATUALIZAÇÃO ====
Média (ms): 463.03 ms
Mínimo (ms): 20.47 ms
Máximo (ms): 963.43 ms
p90 (ms): 752.91 ms
p95 (ms): 822.80 ms

Todos os registros estão consistentes.



In [29]:
resultados_cenario4 = processar_cenario('mongodb.json', 'postgres.json', "Cenário 4 - Req")
print_resultados(resultados_cenario4)

==== RESULTADOS DA COMPARAÇÃO ====

Total de registros comparados: 1442
Registros consistentes: 1442
Registros inconsistentes: 0
Taxa de consistência: 100.00%

==== IGUALDADE DOS CAMPOS ====
clientId: 1442 iguais de 1442 (100.00%)
type: 1442 iguais de 1442 (100.00%)
amount: 1442 iguais de 1442 (100.00%)
description: 1442 iguais de 1442 (100.00%)

==== LATÊNCIA DE CRIAÇÃO ====
Média (ms): 496.41 ms
Mínimo (ms): 17.91 ms
Máximo (ms): 999.70 ms
p90 (ms): 765.21 ms
p95 (ms): 844.67 ms

==== LATÊNCIA DE ATUALIZAÇÃO ====
Média (ms): 497.76 ms
Mínimo (ms): 17.12 ms
Máximo (ms): 993.08 ms
p90 (ms): 793.46 ms
p95 (ms): 843.92 ms

Todos os registros estão consistentes.

